In [1]:
import sys
sys.path.append("/home/srelins/docs/cy_fdm_builder/src")
from testing_helpers import *
from FDMTable import *
from FDMDataset import *

## TO DO:

* PRIORITY: Work our what happens if you re-run .build() on a dataset
* re-running .build() needs "outside_obs" tables to be added back to src tables - TEST THIS


* THEN: build FDM datasets from just dataset_id:
    + summarise tables in dataset init i.e. which are ready for FDM build, which arent, any FDM tables that will be overwritten, any src tables that will be re-combined
    * think about "verbose" functionality of FDMTables - silence init-messages (not required anyways as tables will be initialised from tables in FDM dataset) - "check_build" function
    
    
* add check and warning for Null datetimes when adding start/end dates
* add .drop() method to table
* raw input functionality - copying src tables? overwriting existing tables?
* build data dict function
* ask about requirement to have a raw version of FDM_tables at all times? - read only source?

## Testing:

In [2]:
build_test_environment()

/home/srelins/anaconda3/envs/FDMBuilder/lib/python3.9/site-packages/pandas/core/frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


LOADING: table 1
LOADING: table 2
LOADING: table 3
LOADING: table 4
LOADING: table 5
LOADING: table 6


In [3]:
table_1 = FDMTable(
    source_table_full_id=f"{PROJECT}.CY_TESTS_SRC.src_table_1",
    dataset_id="CY_TESTS_FDM"
)

In [4]:
table_1.quick_build(event_start_date_cols="start_date",
                     event_start_date_format="YMD",
                     event_end_date_cols="end_date",
                     event_end_date_format="YMD")

Building src_table_1:
    src_table_1 copied to CY_TESTS_FDM
    src_table_1 already contains person_id column
    event_start_date column added
    event_end_date column added
Done.


In [5]:
table_2 = FDMTable(
    source_table_full_id = f"{PROJECT}.CY_TESTS_SRC.src_table_2",
    dataset_id = "CY_TESTS_FDM" 
)

In [6]:
table_2.quick_build(
    event_start_date_cols=["start_year", "start_month", "start_day"], 
    event_start_date_format="YMD", 
    event_end_date_cols=["end_year", "end_month", "end_day"], 
    event_end_date_format="YMD"
)

Building src_table_2:
    src_table_2 copied to CY_TESTS_FDM
    person_id column added
    event_start_date column added
    event_end_date column added
Done.


In [7]:
table_3 = FDMTable(
    source_table_full_id=f"{PROJECT}.CY_TESTS_SRC.src_table_3",
    dataset_id="CY_TESTS_FDM"
)

In [8]:
table_3.quick_build(
    event_start_date_cols=["year", "month", "15"], 
    event_start_date_format="YMD", 
)

Building src_table_3:
    src_table_3 copied to CY_TESTS_FDM
    person_id column added
    event_start_date column added
    no event_end_date info provided
Done.


In [9]:
table_4 = FDMTable(
    source_table_full_id=f"{PROJECT}.CY_TESTS_SRC.src_table_4",
    dataset_id="CY_TESTS_FDM"
)

In [10]:
table_4.quick_build(
    event_start_date_cols="start_date_string",
    event_start_date_format="MDY", 
    event_end_date_cols="end_date_string",
    event_end_date_format="DMY"
)

Building src_table_4:
    src_table_4 copied to CY_TESTS_FDM
    src_table_4 already contains person_id column

    70 will be parsed as 2070. Consider converting year.
                
    event_start_date column added

    70 will be parsed as 2070. Consider converting year.
                
    event_end_date column added
Done.


In [11]:
table_5 = FDMTable(
    source_table_full_id=f"{PROJECT}.CY_TESTS_SRC.src_table_5",
    dataset_id="CY_TESTS_FDM"
)

In [12]:
table_5.copy_table_to_dataset()

In [13]:
table_5.head()

,person_id,EDRN,date_string
0,51,KQ,May-2012
1,44,JT,May-2016
2,junk_person_id_642,junk_EDRN_642,July-1968
3,52,KR,July-1993
4,57,KW,June-1973
5,48,JX,June-2002
6,55,KU,April-1976
7,43,JS,March-1986
8,53,KS,March-2003
9,45,JU,January-1954


In [14]:
table_5.add_column("SPLIT(date_string, '-')[OFFSET(0)] AS year")
table_5.add_column("SPLIT(date_string, '-')[OFFSET(1)] AS month")
table_5.head()

,person_id,EDRN,date_string,year,month
0,51,KQ,May-2012,May,2012
1,44,JT,May-2016,May,2016
2,junk_person_id_642,junk_EDRN_642,July-1968,July,1968
3,52,KR,July-1993,July,1993
4,57,KW,June-1973,June,1973
5,48,JX,June-2002,June,2002
6,55,KU,April-1976,April,1976
7,53,KS,March-2003,March,2003
8,43,JS,March-1986,March,1986
9,junk_person_id_780,junk_EDRN_780,January-1986,January,1986


In [15]:
table_5.quick_build(
    event_start_date_cols=["year", "month", "15"],
    event_start_date_format="YMD", 
)

Building src_table_5:
    existing copy of src_table_5 in CY_TESTS_FDM
    src_table_5 already contains person_id column
    event_start_date column added
    no event_end_date info provided
Done.


In [16]:
table_6 = FDMTable(
    source_table_full_id=f"{PROJECT}.CY_TESTS_SRC.src_table_6",
    dataset_id="CY_TESTS_FDM"
)

In [17]:
table_6.copy_table_to_dataset()

In [18]:
table_6.head()

,person_id,digest,EDRN,date_string
0,52,AH,KR,22-Jun-1977
1,56,AL,KV,24-Jan-1960
2,junk_person_id_171,junk_digest_171,junk_EDRN_171,20-May-1999
3,50,AF,KP,11-Feb-1986
4,67,BM,LW,18-Nov-1959
5,54,AJ,KT,1-Sep-1992
6,64,BJ,LT,5-Apr-1982
7,58,AN,KX,12-Dec-1994
8,68,BN,LX,4-Jun-1970
9,junk_person_id_81,junk_digest_81,junk_EDRN_81,26-Oct-1961


In [19]:
table_6.quick_build(
    event_start_date_cols="date_string",
    event_start_date_format="DMY",
)

Building src_table_6:
    existing copy of src_table_6 in CY_TESTS_FDM
    src_table_6 already contains person_id column
    event_start_date column added
    no event_end_date info provided
Done.


In [20]:
dataset = FDMDataset(
    dataset_id = "CY_TESTS_FDM"
)

In [23]:
dataset.build()

		 ##### BUILDING FDM DATASET CY_TESTS_FDM #####
________________________________________________________________________________

1. Checking dataset for source tables:

    src_table_1 contains:  * person_id * event_start_date  * event_end_date
	-> Table ready
    src_table_2 contains:  * person_id * event_start_date  * event_end_date
	-> Table ready
    src_table_3 contains:  * person_id * event_start_date 
	-> Table ready
    src_table_4 contains:  * person_id * event_start_date  * event_end_date
	-> Table ready
    src_table_5 contains:  * person_id * event_start_date 
	-> Table ready
    src_table_6 contains:  * person_id * event_start_date 
	-> Table ready
	Adding problem entries back into tables:
    src_table_1- done
    src_table_2- done
    src_table_3- done
    src_table_4- done
    src_table_5- done
    src_table_6- done

Building person table

    * Person table built with 70 entries

Building initial observation_period table

    * observation_period table built with 90 

In [22]:
blah = "got_money_mo_problems"
blah[:-9]

'got_money_mo'